In [119]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [120]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/kristindong/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache


In [14]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [15]:
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')
slide_elem

<div class="list_text">
<div class="list_date">May 1, 2022</div>
<div class="content_title">NASA's Mars 2020 Rover Tests Descent-Stage Separation</div>
<div class="article_teaser_body">A crane lifts the rocket-powered descent stage away from NASA's Mars 2020 rover after technicians tested the pyrotechnic charges that separate the two spacecraft.</div>
</div>

In [16]:
slide_elem.find('div', class_='content_title')

<div class="content_title">NASA's Mars 2020 Rover Tests Descent-Stage Separation</div>

In [22]:
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"NASA's Mars 2020 Rover Tests Descent-Stage Separation"

In [23]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

"A crane lifts the rocket-powered descent stage away from NASA's Mars 2020 rover after technicians tested the pyrotechnic charges that separate the two spacecraft."

### Featured Images

In [140]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [26]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [27]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [28]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars2.jpg'

In [ ]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

In [30]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [31]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [32]:
browser.quit()

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [227]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'

browser.visit(url)

In [228]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []
img_list=[]

# 3. Write code to retrieve the image urls and titles for each hemisphere.
html = browser.html
hem_soup = soup(html, 'html.parser')

# get the 4 description elements that hold the links for each hemisphere
desc_elem = hem_soup.find_all('div', class_="description")

# loop through list of results
for hem in desc_elem:
    # extract hemisphere relative url and append to base url
    hem_url_rel = hem.find('a', class_="itemLink product-item").get('href')
    hem_url = f'{url}{hem_url_rel}'

    # navigate to hemisphere page and get full image link and title
    browser.visit(hem_url)
    html = browser.html
    img_soup = soup(html, 'html.parser')
    img_url_rel = img_soup.find('img', class_="wide-image").get('src')
    img_url = f'{url}{img_url_rel}'
    title = img_soup.find("h2", class_="title").text
    
    # Add disctionary of image and title to list
    hemisphere_image_urls.append({
        'img_url': img_url,
        'title': title})
    
    browser.back()

In [229]:
# 4. Print the list of dictionaries holding image urls and titles.
hemisphere_image_urls

[{'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [230]:
# 5. Quit the browser
browser.quit()